<a href="https://colab.research.google.com/github/Linda8a/Projeto-otimiza-o-exerc-cio-tempo-e-dieta/blob/main/otimizar%20treino.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog


nombres = [
    'Frango (Grelhado)', 'Ovo (Cozido)', 'Arroz (Cozido)', 'Feijão (Cozido)',
    'Azeite Oliva', 'Cenoura (Crua)', 'Lombo Suíno (Assado)', 'Carne (Coxão Mole)',
    'Pão Francês', 'Batata (Cozida)', 'Queijo (Muzarela)', 'Macarrão (Cru)'
]

costos = np.array([
    0.02259, 0.01333, 0.00439, 0.00539, 0.07587, 0.00415,
    0.02690, 0.05490, 0.01650, 0.00465, 0.04490, 0.00934
])

A_nut = np.array([
    [0.320, 0.133, 0.025, 0.048, 0.000, 0.013, 0.357, 0.324, 0.080, 0.012, 0.226, 0.100],  # proteína
    [0.000, 0.006, 0.281, 0.136, 0.000, 0.077, 0.000, 0.000, 0.586, 0.119, 0.030, 0.779],  # carbo
    [0.025, 0.095, 0.002, 0.005, 1.000, 0.002, 0.064, 0.089, 0.031, 0.000, 0.252, 0.013]   # grasa
])

e_calorias = 4*A_nut[0] + 4*A_nut[1] + 9*A_nut[2]

# BMR
def get_bmr(sexo, peso, altura, edad):
    d = 5 if sexo.lower() == "m" else -161
    return 10*peso + 6.25*altura - 5*edad + d

def get_lambda_and_Y(tempo):
    if tempo < 30: return 1.2, 0, "Insuficiente (<30 min)"
    if tempo <= 75: return 1.55, 4, "Moderado (30–75 min)"
    if tempo <= 120: return 1.725, 9, "Alto (75–120 min)"
    return 1.90, 15, "Extremo (>120 min)"


# OPTIMIZACIÓN DE LA DIETA

def optimizar_dieta(peso, E_target):

    # Macros objetivo
    prot_min, prot_max = 1.6*peso, 2.2*peso
    carb_min, carb_max = 0.45*E_target/4, 0.65*E_target/4
    gras_min, gras_max = 0.20*E_target/9, 0.30*E_target/9
    E_min, E_max = E_target - 50, E_target + 50

    # Restricciones básicas
    A_ub = np.array([
        -A_nut[0], A_nut[0],  # proteína
        -A_nut[1], A_nut[1],  # carbohidratos
        -A_nut[2], A_nut[2],  # grasas
        -e_calorias, e_calorias  # energía
    ])

    b_ub = np.array([
        -prot_min, prot_max,
        -carb_min, carb_max,
        -gras_min, gras_max,
        -E_min, E_max
    ])


    # DIVERSIDAD POR MACRO (50%)

    alpha = 0.50

    A_extra = []
    b_extra = []

    for i in range(len(nombres)):
        # Proteína
        rp = np.zeros(len(nombres)); rp[i] = A_nut[0][i]
        A_extra.append(rp); b_extra.append(alpha * prot_max)

        # Carbo
        rc = np.zeros(len(nombres)); rc[i] = A_nut[1][i]
        A_extra.append(rc); b_extra.append(alpha * carb_max)

        # Grasa
        rg = np.zeros(len(nombres)); rg[i] = A_nut[2][i]
        A_extra.append(rg); b_extra.append(alpha * gras_max)

    A_ub_total = np.vstack([A_ub, np.array(A_extra)])
    b_ub_total = np.concatenate([b_ub, b_extra])

    bounds = [(0, None)] * len(nombres)  # alimentos ilimitados

    res = linprog(costos, A_ub=A_ub_total, b_ub=b_ub_total,
                  bounds=bounds, method="highs")

    return res, prot_min, prot_max, carb_min, carb_max

# ENTRENAMIENTO + DIETA


def correr_calculadora():
    print("\n" + "="*60)
    print("      CALCULADORA DE DIETA ÓPTIMA + ENTRENAMIENTO")
    print("="*60)

    sexo = input("Sexo (m/f): ").strip()
    peso = float(input("Peso (kg): "))
    altura = float(input("Altura (cm): "))
    edad = int(input("Idade (anos): "))
    tempo = int(input("Tempo de treino hoje (min): "))

    # ---- ENTRENAMIENTO ----
    lamb, Y_opt, desc = get_lambda_and_Y(tempo)

    bmr = get_bmr(sexo, peso, altura, edad)
    E_objetivo = bmr*lamb + 300

    # ---- DIETA ----
    res, prot_min, prot_max, carb_min, carb_max = optimizar_dieta(peso, E_objetivo)

    print("\n>>> RESULTADOS DO MODELO ACOPLADO <<<\n")

    # Información de entrenamiento
    print(f"[A] PERFIL DE TREINO ({tempo} min)")
    print(f" - Nivel de Atividade (λ): {lamb}")
    print(f" - Volume Ótimo de Treino (Y): {Y_opt} séries")
    print(f" - Categoria: {desc}")
    print(f" - TDEE Ajustado: {E_objetivo:.0f} kcal/d (+300 kcal)\n")

    if not res.success:
        print("[ERRO] Dieta inviável. Esto es extremadamente raro ahora.")
        return

    # Alimentación
    print("[B] DIETA ÓTIMA ENCONTRADA")
    print(f" - Custo Mínimo Diário: R$ {res.fun:.2f}\n")

    df = pd.DataFrame({"Alimento": nombres, "Gramos": res.x})
    df = df[df["Gramos"] > 1].sort_values("Gramos", ascending=False)
    print(df.round(1).to_string(index=False))

    # Macros
    nut = A_nut @ res.x
    cal = np.dot(e_calorias, res.x)

    print("\n[C] MACROS FINAIS")
    print(f" - Proteína: {nut[0]:.1f} g  (meta: {prot_min:.1f}-{prot_max:.1f})")
    print(f" - Carboidratos: {nut[1]:.1f} g  (meta: {carb_min:.1f}-{carb_max:.1f})")
    print(f" - Gordura: {nut[2]:.1f} g (20–30%)")
    print(f" - Energia Total: {cal:.0f} kcal")

    print("\nModelo executado com sucesso.\n")


# Ejecutar
correr_calculadora()



      CALCULADORA DE DIETA ÓPTIMA + ENTRENAMIENTO
Sexo (m/f): m
Peso (kg): 95
Altura (cm): 180
Idade (anos): 35
Tempo de treino hoje (min): 75

>>> RESULTADOS DO MODELO ACOPLADO <<<

[A] PERFIL DE TREINO (75 min)
 - Nivel de Atividade (λ): 1.55
 - Volume Ótimo de Treino (Y): 4 séries
 - Categoria: Moderado (30–75 min)
 - TDEE Ajustado: 3253 kcal/d (+300 kcal)

[B] DIETA ÓTIMA ENCONTRADA
 - Custo Mínimo Diário: R$ 16.51

            Alimento  Gramos
      Arroz (Cozido)   777.0
        Ovo (Cozido)   570.7
      Macarrão (Cru)   339.3
Lombo Suíno (Assado)    63.7
        Azeite Oliva     8.0

[C] MACROS FINAIS
 - Proteína: 152.0 g  (meta: 152.0-209.0)
 - Carboidratos: 486.0 g  (meta: 365.9-528.6)
 - Gordura: 72.3 g (20–30%)
 - Energia Total: 3203 kcal

Modelo executado com sucesso.

